# Multi-Agent Conversation and Stand-up Comedy

## Imports and dependenices

## Setup and cache model

In [1]:
from utils.model_loader import load_local_hf_model

llm = load_local_hf_model(model_path="meta-llama/Meta-Llama-3-8B-Instruct", stop_token="<|eot_id|>")

/Users/leesean/agentic_workflows/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/leesean/agentic_workflows/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.69s/it]
Device set to use mps:0


## Define a chatbot agent

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful chatbot."),
    HumanMessage(content="Tell me a joke.")
]

response = llm.invoke(messages)
print(response.content)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful chatbot.<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell me a joke.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you smile! Do you want to hear another one?


## Lack of persistence between chats

In [3]:
messages_two = [
    SystemMessage(content="You are a helpful chatbot."),
    HumanMessage(content="Repeat the joke.")
]

response_two = llm.invoke(messages_two)
print(response_two.content)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful chatbot.<|eot_id|><|start_header_id|>user<|end_header_id|>

Repeat the joke.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I apologize, but I didn't tell a joke yet! I'm happy to share one with you if you'd like.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [13]:
from langchain_core.messages import AIMessage
from utils.formatter import extract_response_llama
from rich.pretty import pprint

cathy_messages = [
    SystemMessage(content="Your name is cathy and you are a stand-up comedian."),
    HumanMessage(content="I'm Joe. Cathy, let's keep the jokes rolling.")
]

pprint(cathy_messages)

joe_messages = [
    SystemMessage(content="You name is Joe and you are a stand-up comedian. Start your joke from the punchline of the previous joke."),
    AIMessage(content="I'm Joe. Cathy, let's keep the jokes rolling.")
]

pprint(joe_messages)

[
│   SystemMessage(
│   │   content='Your name is cathy and you are a stand-up comedian.',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   HumanMessage(
│   │   content="I'm Joe. Cathy, let's keep the jokes rolling.",
│   │   additional_kwargs={},
│   │   response_metadata={}
│   )
]

[
│   SystemMessage(
│   │   content='You name is Joe and you are a stand-up comedian. Start your joke from the punchline of the previous joke.',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   AIMessage(
│   │   content="I'm Joe. Cathy, let's keep the jokes rolling.",
│   │   additional_kwargs={},
│   │   response_metadata={}
│   )
]

In [14]:
max_turns = 2

for _ in range(max_turns):
    cathy_response = extract_response_llama((llm.invoke(cathy_messages)).content)
    cathy_messages.append(AIMessage(content=cathy_response))
    joe_messages.append(HumanMessage(content=cathy_response))
    joe_response = extract_response_llama((llm.invoke(joe_messages)).content)
    joe_messages.append(AIMessage(content=joe_response))
    cathy_messages.append(HumanMessage(content=joe_response))


In [15]:
pprint(joe_messages)

[
│   SystemMessage(
│   │   content='You name is Joe and you are a stand-up comedian. Start your joke from the punchline of the previous joke.',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   AIMessage(
│   │   content="I'm Joe. Cathy, let's keep the jokes rolling.",
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   HumanMessage(
│   │   content='Joe! Great to be here! So, I was at the park the other day, and I saw a guy playing fetch with his dog. But here\'s the thing, Joe – the dog wasn\'t even bringing the ball back! He was just running around, sniffing things, having the time of his life. I was like, "Dude, you\'re getting played! You\'re just a glorified Uber driver for this dog!" (laughs)',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   AIMessage(
│   │   content='That\'s a great one! You know, I was at the vet the other day, and I saw a dog that was so old, it was like it was trying to get a senior discount on its flea treatment. I was like, "Dude, you\'re not even getting a discount, you\'re just getting a free ride to the other side!" (laughs)',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   HumanMessage(
│   │   content='(laughs) Oh man, that\'s perfect! I love it! You know, I was at the grocery store the other day, and I saw this sign that said, "10 items or less." And I\'m thinking, "No, it\'s 10 items or FEWER, you\'re not trying to give me a participation trophy for shopping!" (laughs)\n\nAnd speaking of shopping, have you ever noticed how some stores have those signs that say, "Please do not return to the store with your cart"? Like, what\'s the worst that\'s going to happen? You\'re going to come back with a cart full of stolen goods and a "I\'m with stupid" t-shirt? (laughs)',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   AIMessage(
│   │   content='(laughs) Ahahaha, oh man, that\'s great! I love the "I\'m with stupid" t-shirt idea! You know, I was at the bank the other day, and I saw a sign that said, "Please do not make large withdrawals." And I\'m thinking, "What, are you worried I\'m going to withdraw a small elephant or something?" (laughs)\n\nAnd have you ever noticed how some restaurants have those signs that say, "Please wait to be seated"? Like, what\'s the alternative? Just walk in and start eating off the buffet table? "Hey, I\'ll just take a plate of nachos and a side of awkwardness, please!" (laughs)',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   )
]

In [9]:
pprint(cathy_messages)

[
│   SystemMessage(
│   │   content='Your name is cathy and you are a stand-up comedian.',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   HumanMessage(
│   │   content="I'm Joe. Cathy, let's keep the jokes rolling.",
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   HumanMessage(
│   │   content='And then I thought, "You know who\'s even worse off? The guy who\'s paying for that Uber ride – his therapist! \'I\'m feeling anxious, I\'m feeling depressed, I\'m feeling like I\'m just a glorified ball-fetcher for my dog\'s Instagram followers!\' (laughs)',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   ),
│   HumanMessage(
│   │   content='(laughs) Ahahah, that\'s amazing! You know, I was at a fancy restaurant the other day, and the waiter comes over and says, "And for dessert, we have our signature crème brûlée." And I\'m like, "Oh, great, because what I really needed was a rich, creamy custard base topped with a layer of caramelized sugar, served in a delicate ceramic cup and garnished with a sprinkle of edible gold dust." No, I just wanted a spoon and a napkin, buddy! (laughs)',
│   │   additional_kwargs={},
│   │   response_metadata={}
│   )
]